In [18]:
# Importando as bibliotecas
import pandas as pd

In [19]:
# Ler as 31 primeiras linhas como strings
with open("../data/103_B.csv", "r", encoding="utf-8") as f:
    header_lines = [next(f).strip() for _ in range(31)]

In [20]:
# Criar dicionário com 5 informações que você quer
informacoes_interessantes = ["INTERVAL", "NOMINAL VOLTAGE", "FREQUENCY", "START", "STOP"]
metadados = {}

In [21]:
with open(file_path, "r", encoding="utf-8") as f:
    for _ in range(31):  # primeiras 31 linhas do cabeçalho
        line = f.readline().strip()
        if ";" in line:
            # separa pelos primeiros 2 ";" para pegar Coluna A e C
            parts = line.split(";")
            key = parts[0].strip()
            value = parts[2].strip()  # Coluna C
            if key in informacoes_interessantes:
                metadados[key] = value
                
# Limpar aspas simples de todos os valores
metadados = {k: v.strip("'") for k, v in metadados.items()}

print(metadados)


# Converter START e STOP para datetime diretamente no dicionário
metadados['START'] = pd.to_datetime(metadados['START'], format="%Y/%m/%d_%H:%M:%S")
metadados['STOP'] = pd.to_datetime(metadados['STOP'], format="%Y/%m/%d_%H:%M:%S")

# Calcular duração e adicionar ao dicionário
metadados['DURATION'] = metadados['STOP'] - metadados['START']

# Mostrar resultado
for k, v in metadados.items():
    print(f"{k}: {v}")

{'INTERVAL': '30 sec.', 'NOMINAL VOLTAGE': '220V', 'FREQUENCY': '60Hz', 'START': '2023/06/19_14:44:45', 'STOP': '2023/06/29_18:35:04'}
INTERVAL: 30 sec.
NOMINAL VOLTAGE: 220V
FREQUENCY: 60Hz
START: 2023-06-19 14:44:45
STOP: 2023-06-29 18:35:04
DURATION: 10 days 03:50:19


In [48]:
# Caminho para o arquivo
file_path = "../data/103_B.csv"

# Lendo todo o CSV como tabela genérica
df = pd.read_csv(file_path, sep=";", header=None)

# Selecionar linhas 33 a 37 (em pandas o índice começa em 0, então é 32:37)
df_tensoes = df.loc[[32, 33, 35], [3, 4, 5]].copy()

# Nomeando colunas
df_tensoes.columns = ["V1", "V2", "V3"]

# Nomeando as linhas (conforme a ordem que você passou)
df_tensoes.index = [
    "Average value",
    "Max value",
    "Min value",
]

# Converter todas as colunas do dataframe
df_tensoes = df_tensoes.astype(str)                # garantir que são strings
df_tensoes = df_tensoes.apply(lambda x: x.str.replace(",", "."))
df_tensoes = df_tensoes.apply(pd.to_numeric, errors='coerce')

print(df_tensoes)

limite_min = 207
limite_max = 253

print(f"\nLimite mínimo: {limite_min}\nLimite máximo: {limite_max}")

tensao_max = df_tensoes.loc["Max value"]
tensao_min = df_tensoes.loc["Min value"]

for col in df_tensoes.columns[0:3]:
    if tensao_min[col] < limite_min:
        print(f"{col}: valor mínimo {tensao_min[col]} V abaixo do limite mínimo ({limite_min} V)")
    if tensao_max[col] > limite_max:
        print(f"{col}: valor máximo {tensao_max[col]} V acima do limite máximo ({limite_max} V)")
    if limite_min <= tensao_min[col] <= limite_max and limite_min <= tensao_max[col] <= limite_max:
        print(f"{col}: tensões dentro do limite.")

                  V1     V2     V3
Average value  222.0  221.0  223.0
Max value      235.0  235.0  235.0
Min value      213.0  211.0  213.0

Limite mínimo: 207
Limite máximo: 253
V1: tensões dentro do limite.
V2: tensões dentro do limite.
V3: tensões dentro do limite.
